# Baseline Chat Interface
> A standard, baseline, direct implementation of the "basic user interface". 

In this notebook, we create the simplest baseline implementation of the "basic user interface". The chat is implemented directly through langchain/openAI, where the prompt is directly input into langchain instead of using the ai_classroom_suite module. 

In [ ]:
## run this code only if you're local and developing
import os, sys

# get parent of current working directory
parent_dir = os.path.dirname(os.getcwd())

#append to path
sys.path.append(parent_dir)

In [ ]:
#| to_script

import os
import openai
import gradio as gr
import pandas as pd

## Interface Helpers and Functionality

In the next section, we'll create some helper functions to make sure we're able to create the interface we want, and that it behaves nicely. 

In [ ]:
#| to_script

# The OpenAI API key provided by the instructor
openai.api_key = os.environ.get("OPENAI_API_KEY")

# Instructor's prompt (invisible to students)
instructor_prompt = os.environ.get("SECRET_PROMPT")

In [ ]:
#| to_script

# Adds a message from the user to the conversation history
def add_user_message(msg, history, messages):
    messages.append({"role": "user", "content": msg})
    return "", history + [[msg, None]], messages

# Generate a response from the chatbot
def get_tutor_reply(history, messages):
    completion = openai.chat.completions.create( 
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=150 # Change this 
    ) 
    # Extract the model's reply
    reply = completion.choices[0].message.content 
    # Updates the conversation history with the chatbot's response
    history[-1][1] = reply
    messages.append({"role": "assistant", "content": reply})
    return history, messages

In [ ]:
#| to_script

# Saves the conversation history to a JSON file.
def save_chat_to_json(history):
    formatted_convo = pd.DataFrame(history, columns=['user', 'chatbot'])
    output_fname = f'tutoring_conversation.json'
    formatted_convo.to_json(output_fname, orient='records')
    return gr.update(value=output_fname, visible=True)

## The User Interface

Below, we put all of this information together with the actual formatting of the user interface.

### Note for Instructors ###

You need to provide an OpenAI API Key as well as a Secret Prompt to initialize the model. 

- To set the API Key, in the hosted app on Huggingface Space, go to ``Settings -> Variables and Secrets -> Secrets``, then replace ``OPENAI_API_KEY`` value with your key.
If you haven't created one already, visit [platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys) to sign up for an account and get your personal API key. 

- To set the Secret Prompt, in the hosted app on Huggingface Space, go to ``Settings -> Variables and Secrets -> Secrets``, then replace ``SECRET_PROMPT`` value with your key.

In [ ]:
#| to_script

### User Interfaces ###
with gr.Blocks() as SimpleChat:
    gr.Markdown("""
        ## Chat with A Tutor
        Description here
        """)
    
    # Initialize the model with system message given by the instructor prompt
    system_msg = [{"role": "system", "content":  instructor_prompt}]
    messages = gr.JSON(visible=False, value = system_msg)
    
    with gr.Group():
        chatbot = gr.Chatbot(label="Simple Tutor")
        with gr.Row(equal_height=True):
            user_chat_input = gr.Textbox(show_label=False, scale=9)
            submit_btn = gr.Button("Enter", scale=1)
    with gr.Group():
        export_dialogue_button_json = gr.Button("Export your chat history as a .json file")
        file_download = gr.Files(label="Download here", file_types=['.json'], visible=False)
    
    # User submits a message by either click the submit button or press enter on keyboard
    submit_btn.click(
        fn=add_user_message, inputs=[user_chat_input, chatbot, messages], outputs=[user_chat_input, chatbot, messages], queue=False
    ).then(
        fn=get_tutor_reply, inputs=[chatbot, messages], outputs=[chatbot, messages], queue=True
    )
    user_chat_input.submit(
        fn=add_user_message, inputs=[user_chat_input, chatbot, messages], outputs=[user_chat_input, chatbot, messages], queue=False
    ).then(
        fn=get_tutor_reply, inputs=[chatbot, messages], outputs=[chatbot, messages], queue=True
    )
    export_dialogue_button_json.click(save_chat_to_json, chatbot, file_download, show_progress=True)

In [ ]:
#| to_script
SimpleChat.queue().launch(server_name='0.0.0.0', server_port=7860)

A little helper in case your ports are open and you just want to close them all. If this doesn't work, restart your IDE.

In [ ]:
gr.close_all()